## Steelers Pre-Game Tweets
**By:** _Mike Scheibel_

In [2]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import os
import time
from Mike_API_Keys import api_key, api_key_secret, access_token, access_token_secret
from datetime import datetime
#from scraping_tweets import scraptweets

In [3]:
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                    'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                    'retweetcount', 'text', 'hashtags'])

In [5]:
search_words = "#HereWeGo OR #Steelers"
date_since = "2020-11-19"
numTweets = 1000
numRuns = 100

In [6]:
for i in range(0, numRuns):
    # We will time how long it takes to scrape tweets for each run:
    start_run = time.time()

    # Collect tweets using the Cursor object
    # .Cursor() returns an object that you can iterate or loop over to access the data collected.
    # Each item in the iterator has various attributes that you can access to get information about each tweet
    tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)

    # Store these tweets into a python list
    tweet_list = [tweet for tweet in tweets]

    # Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # tweet.entities['hashtags'] - hashtags in the tweet

    # Begin scraping the tweets individually:
    noTweets = 0

    for tweet in tweet_list:

        # Pull the values
        username = tweet.user.screen_name
        acctdesc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        usercreatedts = tweet.user.created_at
        tweetcreatedts = tweet.created_at
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']

        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            text = tweet.full_text

        # Add the 11 variables to the empty list - ith_tweet:
        ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                     usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

        # Append to dataframe - db_tweets
        db_tweets.loc[len(db_tweets)] = ith_tweet

        # increase counter - noTweets  
        noTweets += 1

    # Run ended:
    end_run = time.time()
    duration_run = round(end_run-start_run, 2)

    print('no. of tweets scraped for run {} is {}'.format(i, noTweets))
    print('time take for {} run to complete is {}'.format(i, duration_run))

    time.sleep(30) #30second sleep time


no. of tweets scraped for run 0 is 1000
time take for 0 run to complete is 54.95
no. of tweets scraped for run 1 is 1000
time take for 1 run to complete is 54.88
no. of tweets scraped for run 2 is 1000
time take for 2 run to complete is 755.71
no. of tweets scraped for run 3 is 1000
time take for 3 run to complete is 55.51
no. of tweets scraped for run 4 is 1000
time take for 4 run to complete is 56.08
no. of tweets scraped for run 5 is 1000
time take for 5 run to complete is 720.34
no. of tweets scraped for run 6 is 1000
time take for 6 run to complete is 57.57
no. of tweets scraped for run 7 is 1000
time take for 7 run to complete is 57.32
no. of tweets scraped for run 8 is 1000
time take for 8 run to complete is 717.87
no. of tweets scraped for run 9 is 1000
time take for 9 run to complete is 59.99
no. of tweets scraped for run 10 is 1000
time take for 10 run to complete is 750.89
no. of tweets scraped for run 11 is 1000
time take for 11 run to complete is 60.38
no. of tweets scrape

no. of tweets scraped for run 99 is 1000
time take for 99 run to complete is 546.72


In [7]:
db_tweets

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags
0,BrayGod,the white Wesley Snipes,Area 51,435,466,14100,2012-06-30 04:48:35,2020-11-22 04:44:45,274,Facts! #Steelers https://t.co/TASbwulZsQ,"[{'text': 'Steelers', 'indices': [26, 35]}]"
1,elviabloop,ucla’24 (⁎⁍̴̛ᴗ⁍̴̛⁎),Earthland,343,120,2687,2016-06-23 04:45:25,2020-11-22 04:44:36,274,Facts! #Steelers https://t.co/TASbwulZsQ,"[{'text': 'Steelers', 'indices': [26, 35]}]"
2,teemeeboy21,,,178,133,3463,2014-06-18 12:34:04,2020-11-22 04:43:25,490,MOOD! #Steelers https://t.co/DBlYuddYFk,"[{'text': 'Steelers', 'indices': [24, 33]}]"
3,DerikMcCourtney,el camino driver,,141,49,746,2014-05-25 01:32:50,2020-11-22 04:43:13,29,Stay tuned because tomorrow we will be giving ...,[]
4,iGotGasFYTB,People Change🌓Money Remains The Same NLMB💜📲 Fo...,,2024,1505,39742,2017-12-16 05:45:42,2020-11-22 04:43:11,362,"Landed in Jacksonville 🛬 , back at it tomorrow...","[{'text': 'HereWeGo', 'indices': [83, 92]}]"
...,...,...,...,...,...,...,...,...,...,...,...
99995,PittpanthersML,University of Pittsburgh alum. Pitt season tic...,"Pittsburgh, PA",4101,1844,43367,2010-11-01 18:34:43,2020-11-22 11:57:47,0,Good morning Steeler Nation! Let’s Go! #Here...,"[{'text': 'HereWeGo', 'indices': [41, 50]}]"
99996,RealMcMinn,Huge Pittsburgh Steelers fan. WWE disciple. ...,"Mississippi, USA",241,15,1048,2018-05-28 21:26:21,2020-11-22 11:57:23,0,@AlannaWoolridge Its ass whooping day!!\n#Here...,"[{'text': 'HereWeGo', 'indices': [40, 49]}]"
99997,pierce_delray,"steelers 4 life , love the city ,pull for the ...","Virginia, USA",1324,790,12486,2015-05-04 15:35:12,2020-11-22 11:55:21,35,"Happy birthday to the young goat JuJu, a team ...","[{'text': 'Steelers', 'indices': [75, 84]}]"
99998,AustinHavlin,Bellarmine 2019. RRT at St. Elizabeth.,"Southgate, KY",528,133,2113,2014-02-16 02:32:31,2020-11-22 11:54:57,1201,The #Steelers play the Jags on Sunday so let's...,"[{'text': 'Steelers', 'indices': [23, 32]}]"


In [8]:
# Once all runs have completed, save them to a single csv file:    
# Obtain timestamp in a readable format:

to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')

# Define working path and filename
path = os.getcwd()
filename = to_csv_timestamp + '_steelers_tweets.csv'

# Store dataframe in csv with creation date timestamp
db_tweets.to_csv(filename, index = False)

print('Scraping has completed!')

Scraping has completed!


In [10]:
import nltk
import numpy as np

from string import punctuation
from collections import Counter

from pprint import pprint # get some prettier printing of objects

from nltk.corpus import stopwords

sw = set(stopwords.words('english'))

In [12]:
sd = open("20201122_070537_steelers_tweets.csv", encoding='utf-8').read()

sd_clean = [w for w in sd.lower().split()]
sd_clean = [w.lower() for w in sd_clean if w.isalpha() and w not in sw]

In [13]:
total_tokens = len(sd_clean)
unique_tokens = len(set(sd_clean))
lex_diversity = len(set(sd_clean))/len(sd_clean)
avg_token_len = np.mean([len(w) for w in sd_clean])
top_20 = Counter(sd_clean).most_common(20)

results = {'tokens':total_tokens,
            'unique_tokens':unique_tokens,
            'avg_token_length':avg_token_len,
            'lexical_diversity':lex_diversity,
            'top_20':top_20}

results

{'tokens': 831459,
 'unique_tokens': 4412,
 'avg_token_length': 5.642867537665718,
 'lexical_diversity': 0.005306335008701571,
 'top_20': [('game', 24144),
  ('steelers', 21664),
  ('officially', 18799),
  ('back', 12978),
  ('jacksonville', 12022),
  ('follow', 9840),
  ('fan', 8489),
  ('away', 7968),
  ('play', 7470),
  ('sunday', 6914),
  ('win', 6677),
  ('love', 6625),
  ('jags', 6615),
  ('retweet', 6564),
  ('give', 6404),
  ('making', 6245),
  ('juju', 6241),
  ('catch', 6204),
  ('help', 6008),
  ('awesome', 5936)]}